## Sample use case of a script that automatically gets the Likert Scale tables

This is some rough code that I typed up after class. It is my hope that it can be of some use to people in EE 400.

In your implementation just change the `path` variable to the appropriate path, and then specify the observation filenames in `filepaths`.

- https://stackoverflow.com/questions/10366596/how-to-read-contents-of-an-table-in-ms-word-file-using-python/33775294#33775294

In [1]:
import numpy as np               # All-round useful
import win32com.client as win32  # Necessary to get the Word docs
import pandas as pd              # Our main library to work with tabular data
import re                        # Used to find patterns in text for processing

In [2]:
# Helper Function 1
def getData(table):
    '''
    Assumes: tables is take from a doc = word.ActiveDocument object
    Returns a delightful surprise: our data automatically formatted into beautiful DataFrame'''
    
    # Initialize
    data = [] 
    
    # Loop through all the rows and columns of the word docx table
    for j in range(1, len(table.Columns)+1): 
        col = [] # Initialize a column to fill up
        for i in range(1, len(table.Rows)+1):
            try:
                text = table.Cell(Row =i, Column =j).Range.Text 
                text = text.replace('\r\x07','') # Get rid of a weird mistranslated character
                text = text.replace('\r\x1f','')
                if j in range(2,len(table.Columns)) and i in range(2,len(table.Rows)):
                    text = text.replace(' ','')                
                col.append(text) # add our text data to the column
            except:
                continue # if an exception is encountered just keep calm and carry on
        data.append(col)
    
    data = np.asarray(data).transpose() # Reorient the data to the original format
    return pd.DataFrame(data)

# Helper Function 2
def makeNums(df):
    '''
    Assumes dataframe is a pandas DataFrame with text values in the Likert Scale table
    Returns a pandas DataFrame with numerical values in the Likert Scale table:
            'Not at all' = 1
            'Somewhat' = 2
            'Satisfactory' = 3
            'Outstanding' = 4
    For details see:
    https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html
    '''  
    df = df.replace(to_replace=r'X', value=1, regex=True).replace('\rX',1) # Replace entries with X's with 1s 
    df = df.replace('',0) 
    df.columns = df.iloc[0] # Label the columns correctly
    df = df.drop(df.index[0]) # delete the redundant 0th row
    
    for i in range(1, len(list(df.columns))):    
        df.iloc[:, i] = df.iloc[:, i].replace(1,i) # gives the correct likert value to a column: 1,2,3,4
        
    return df

In [13]:
word = win32.Dispatch("Word.Application") # Not sure what this is but we need it
word.Visible = 0 # Not sure what this is but we need it

# Add the filepath as a string for convenience
path = "C:\\Users\\victo\\Desktop\\Observations-Project\\data\\"

filepaths = [path + "Observations_Form - AMATH 581.docx",
             path + "Observations_Form - PHYS 116.docx",
             path + "Observations_Form - PHYS 123 Waves.docx",
             path + "Observations_Form - PHYS 114 General Physics.docx",
             path + "Observations_Form - CSE 421 Intro to Algorithms.docx",
             path + "Observations_Form - ME 123 Visualization and CAD.docx",
             path + "Observations_Form - ECE 235 Continuous Time Linear Systems.docx"]

# Sample case for "ECE 235 Continuous Time Linear Systems.docx"          
# You might want to write a loop to go through everything and analyze the data

word.Documents.Open(filepaths[6]) 
doc = word.ActiveDocument # Hold the tables in the word doc we're looking at

In [8]:
doc.Tables.Count # Returns the number of tables in 

4

In [9]:
instr1 = doc.Tables(1)   # 1st instructor table 
instr2 = doc.Tables(2)   # 2nd instructor table
students = doc.Tables(3) # Student table

In [10]:
getData(instr1) # Let's look at the output of what we have so far

,0,1,2,3,4
0,During the Observed Session:,Not at All,Somewhat,Satisfactory,Outstanding
1,The instructor provides objectives for the cla...,,X(Therewasapromiseofsomeapplications),,
2,The instructor relates the session content to ...,X,,,
3,The instructor uses visual aids that are clear...,,,,X (Nice slides)
4,"The instructor uses practical, “real-world” ex...",,,,X (Many examples were discussed)
5,The instructor refers to the relevant portions...,X,,,
6,The instructor uses humor effectively to promo...,X,,,
7,The instructor answers questions well and demo...,,,,X (Succinct and clear answers given)
8,The instructor shows clear interest or enthusi...,,,X(Enthusiasticfaceandclearlyinterestedteaching...,
9,The instructor uses student names.,X,,,


In [7]:
# Now we'll use the helper functions to tidy up the table and convert the entries to integers
df1 = getData(instr1)
df1 = makeNums(df1)  

df2 = getData(instr2)
df2 = makeNums(df2)

In [8]:
df_instr = pd.concat([df1,df2]).transpose() # Combine the instructor tables

# Fixes the labels for the instructor ratings. The concatenation resulted in 1-17, 1-13 instead of 1-30
df_instr.columns = list(range(1,31)) # Relabel column names
df_instr

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,,,,,,,,,,,,,,,,,,,,,
During the Observed Session:,The instructor provides objectives for the cla...,The instructor relates the session content to ...,The instructor uses visual aids that are clear...,"The instructor uses practical, “real-world” ex...",The instructor refers to the relevant portions...,The instructor uses humor effectively to promo...,The instructor answers questions well and demo...,The instructor shows clear interest or enthusi...,The instructor uses student names.,The instructor asks specific questions.,...,The instructor could be easily heard.,The instructor moves around in the classroom a...,The instructor uses analogies or metaphors to ...,The instructor emphasizes key points throughou...,The instructor makes eye contact with students.,The instructor uses open (not closed) body lan...,The instructor engages in behaviors that devel...,The instructor relates the material/concepts t...,The instructor is available before class.,The instructor is available after class.
Not at All,0,1,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Somewhat,2,0,0,0,0,0,0,0,0,0,...,0,0,0,2,2,2,2,0,0,0
Satisfactory,0,0,0,0,0,0,0,3,0,3,...,0,3,3,0,0,0,0,0,0,0
Outstanding,0,0,4,4,0,0,4,0,0,0,...,4,0,0,0,0,0,0,4,4,4


In [9]:
df_stud = getData(students)
df_stud = makeNums(df_stud).transpose()  
df_stud

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,,,,,,,,,,,,,
During the Observed Session:,Students maintain attention toward the instruc...,Students remain awake and alert during the obs...,Students are using their cell phones or other ...,Students are over one minute late to class.,Students pack up early at the end of class.,Students are reading the newspaper or doing ot...,Students interact with the instructor before c...,Students interact with the instructor after cl...,Students initiate questions.,Students respond to questions posed by the ins...,Students ask follow up questions.,Students participate in class when asked to do...,Students are taking notes.
None (0%),0,0,0,0,0,0,0,0,0,0,1,0,0
Few,0,0,0,0,0,2,2,0,2,0,0,0,0
Some,0,0,3,0,0,0,0,3,0,3,0,3,0
Many,4,0,0,4,4,0,0,0,0,0,0,0,4
Most (> 50%),0,5,0,0,0,0,0,0,0,0,0,0,0


### Cleaned Data: 

- Your 43 (instructor and student) Likert-Scale Items should be listed in the spreadsheet (one item per column). 


- Name each item with a short label that is referenced in the codebook (to be included as an appendix in the .pdf document).

    - The codebook simply explains anything an individual outside of your group would need to know to understand the data. 
    
    - Any information that you expect to analyze should be coded numerically and explained in the codebook.
    
    - For example, you may want to analyze your data by observer to understand differences in the ratings of observers -- thus, you might code your observers as 1, 2, and 3, rather than by their names.


- The first few columns of your cleaned data should also include relevant information about the observation (date of observation, length of observation, class observed, instructor name, # enrolled students, # students attending, observer, etc..).   

Note that Likert Scale items can simply be coded as 1, 2, 3, 4 corresponding to the four categories in your instructor ratings and as 1, 2, 3, 4, 5 corresponding to the five categories in your student ratings.   

### Descriptive Statistics:   

- This section of your .pdf document should be a table of mean, standard deviation, maximum, and minimum values of all relevant items in the cleaned data (the 43 instructor and student items, enrollment, attendance, etc.).   


- The descriptive statistics should also include mean, standard deviation, maximum, and minimum values of any scales that you select in subsequent analyses of the data (**a scale is an average of several items determined by exploratory factor analyses or similar statistical analysis technique to contain items that measure the same thing**).   

In [10]:
df_instr.describe() # Not exactly what we want. This happened because of the transpose, so we'll need additional code to fix

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
count,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
unique,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
top,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
freq,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [11]:
df_stud.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13
count,6,6,6,6,6,6,6,6,6,6,6,6,6
unique,3,3,3,3,3,3,3,3,3,3,3,3,3
top,0,0,0,0,0,0,0,0,0,0,0,0,0
freq,4,4,4,4,4,4,4,4,4,4,4,4,4


### Exploratory Factor Analyses and Face Validity:   

In this level of analysis, the 43 instructor and student items are first divided into sets of items that have face validity -- face validity implies that, at face value, the items appear to be related to one another and measuring something similar.   

You will likely analyze all student items as one set and the instructor items as no more than two sets of items.   

Tasks:

- Use a principal components analysis on each set of data and take a look at the loadings to determine which items should be grouped into scales. 
    
    - https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60   

    - A rule of thumb is that items that load onto one factor at 0.5 or higher but do not load on another factor at 0.5 or higher can be grouped together into a scale. 
    

- The outcome of your exploratory factor analyses should be a brief paragraph describing which items you retained and which items you discarded (items that did not load well onto any factor or that cross-loaded onto multiple factors are typically discarded) and the names of the scales you decided on for your retained items.   


- Once you decide which items fit into which scales, take the average of the items to get the scale value and add the mean, standard deviation, maximum, and minimum to your descriptive statistics table.   

### Inter-Rater Reliability:   

For those observations that you completed as a group (i.e. where the members of your group observed the same class session)

Tasks :

- Compute the inter-rater reliability using either Kendall's Tau or Spearman's correlation coefficient (available in several software packages including Excel and Matlab and can also be coded in Python) between each pair of observers (if you have three people in your group, you will end up with three pairs to analyze). 


- Analyze the inter-rater reliability of your instructor items separately from your student items.   


- Tabulate your results and discuss the results of your analyses as a group and write up the results of your discussion addressing such questions as --  Did the rubrics help you to be more consistent?  What could be modified to improve the reliability of your rating system?   

Here is an example of Kendall's Tau and how it applies to the inter-rater reliability problem https://www.statisticshowto.datasciencecentral.com/kendalls-tau/